## Playing with sampling and the Central Limit Theorem

Begin with imports:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Let's create a population that isn't normally distributed. We will concatenate several normal distributions to do so:

(Normal Distribution: $p(x) = \frac{1}{\sqrt{ 2 \pi \sigma^2 }} e^{ - \frac{ (x - \mu)^2 } {2 \sigma^2} }$)

In [ ]:
d1 = np.random.normal(loc=-6.4, scale=1.2, size=40000)
d2 = np.random.normal(loc=4, scale=10, size=16000)
d3 = np.random.normal(loc=22, scale=8, size=72000)
population = np.concatenate([d1, d2, d3])
pop = pd.DataFrame(data=population, columns=['population'])
pop.head()

## Make a histogram. Play around with bin size

Hint: there are multiple ways to do this. Try numpy.histogram or the pandas method hist.

In [ ]:
pop['population'].hist(bins=100)

Extra: Try displaying the data using an alternate visualization technique, a violin plot. Seaborn has a built-in method that is useful for this.

In [ ]:
import seaborn as sns
ax = sns.violinplot(y='population', data=pop)

## Make a kernel density estimate of the population distribution

Hint: pandas.DataFrame.plot.kde

In [ ]:
pop.plot.kde()

## Compute the mean and median of the population

Compute mean

In [ ]:
pop['population'].mean()

In class exercise: Compute median

## Computer the range and standard deviation of the population

In class exercise:

## We have described our population. Now let's draw a sample of size n and look at the distrubtion of our sample mean and s.d.

Write a function that samples the pop dataframe with an argument n that is the number of samples to take. Sample without replacement.

In [ ]:
def draw_sample(pop, n):
    subset = np.random.choice(np.array(list(pop.index)), size=n, replace=False)
    sample = pd.DataFrame(data=pop['population'][subset].values, columns=['sample'])
    return sample

In [ ]:
sample = draw_sample(pop, 20)

In [ ]:
sample

## Now we want to draw repeated samples of size *n* from the population

Create another function that calls the first `samples` times. Have `samples` be an argument to the function along with n which is the argument to the first function. For each sample, append the mean and the standard deviation of the sample to two separate lists and return them.

Hint: use a loop with    range(samples) iterations. To create an empty list at the start of a function, try something like:

    def repeat_samples(samples, n):  
      means = []  
      sds = []  
      ...  
      return (means, sds)
    
then use the append method to append each mean and sd value to the end of each respective list.

In [ ]:
def repeat_samples(pop, samples, n):
    means = []
    sds = []
    
    for i in range(samples):
        sample = draw_sample(pop, n)
        means.append(sample['sample'].mean())
        sds.append(sample['sample'].std())
    
    return (means, sds)

In [ ]:
means, sds = repeat_samples(pop, 30, 30)

In [ ]:
means

## Almost there!

Now make a function with two arguments `samples` and `n` that takes the return values from the last function and
* converts the lists to a single dataframe
* plots two histograms of the columns (mean, sd)
* prints out the mean and sd of the columns

Hint: to get a multi-valued return into new variables, try this:

    means, sds = repeat_samples(samples, n)
    df = pd.DataFrame(data={'means: means, 'sds': sds})

In [ ]:
def describe_sample(pop, samples, n):
    means, sds = repeat_samples(pop, samples, n)
    df = pd.DataFrame(data={'means': means, 'sds': sds})
    
    df.hist(bins=100)
    print('Mean: {}'.format(np.round(df['means'].mean(), 2)))
    print('Std Dev: {}'.format(np.round(df['sds'].mean(), 2)))
    
    return df

In [ ]:
df = describe_sample(pop, 1000, 100)

In [ ]:
df = describe_sample(pop, 5000, 100)

## Run your final function several times with varying values of samples and n

How did your result begin to converge on the population mean and sd?

## Bootstrapping your data: Finding confidence intervals

Statisticians take advantage of the central limit theorem as a method of establishing confidence intervals. Create a function that finds the nth and (100-n)th percentiles of the distribution of means found with describe_sample.

In [ ]:
def bootstrapping(pop, sample, n, percentile):
    df = describe_sample(pop, sample, n)
    li = df['means'].quantile(q=percentile)
    ui = df['means'].quantile(q=1-percentile)
    mean = df['means'].mean()
    
    print('Mean: {}: and CI: {} - {}'.format(np.round(mean, 2),
                                             np.round(li, 2), np.round(ui, 2)))
    
    return df, mean, ui, li

In [ ]:
df, mean, ui, li = bootstrapping(pop, 2000, 100, 0.05)